In [3]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import sys
import subprocess

# ----------------------------
# Ensure 'tabulate' is installed for pandas to_markdown()
# ----------------------------
try:
    import tabulate
except ImportError:
    print("Installing 'tabulate' library...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tabulate"])
    import tabulate

# ----------------------------
# Load dataset
# ----------------------------
csv_file = "C:/Users/User/Desktop/Nairobi House Price Prediction/data/raw_listings.csv"
df = pd.read_csv(csv_file)

# Markdown file to save the data dictionary
md_file = "dictionary.md"

with open(md_file, "w", encoding="utf-8") as f:

    # ----------------------------
    # Dataset Overview
    # ----------------------------
    total_listings = len(df)
    collection_date = pd.Timestamp.today().strftime('%B %d, %Y')
    locations_covered = df['location'].nunique()
    property_type_counts = df['property_type'].value_counts().to_dict()

    f.write("# Nairobi House Price Data Dictionary\n\n")
    f.write("## Dataset Overview\n")
    f.write(f"- Total Listings: {total_listings} properties\n")
    f.write(f"- Collection Date: {collection_date}\n")
    f.write(f"- Locations Covered: {locations_covered} Nairobi neighborhoods\n")
    f.write(f"- Property Types: {', '.join([f'{k} ({v})' for k,v in property_type_counts.items()])}\n\n")

    # ----------------------------
    # Column Descriptions
    # ----------------------------
    f.write("## Column Descriptions\n\n")
    f.write("| Column | Type | Examples | Stats / Notes |\n")
    f.write("|--------|------|---------|---------------|\n")

    columns = ['location', 'property_type', 'bedrooms', 'bathrooms', 'size_sqft', 
               'amenities', 'price_kes', 'listing_date']

    for col in columns:
        if col not in df.columns:
            continue

        col_type = df[col].dtype
        examples = df[col].dropna().unique()[:5]
        missing = df[col].isnull().sum()
        stats_note = ""

        if col_type in ['int64', 'float64']:
            stats_note = f"Missing: {missing}, Range: {df[col].min()} - {df[col].max()}"
            if col == 'price_kes':
                stats_note += f", Mean: {df[col].mean():.2f}, Median: {df[col].median():.2f}"
            if col == 'size_sqft':
                stats_note += ", Outliers possible"
        else:
            stats_note = f"Missing: {missing}, Unique: {df[col].nunique()}"

        examples_str = ', '.join(map(str, examples))
        f.write(f"| {col} | {col_type} | {examples_str} | {stats_note} |\n")

    # ----------------------------
    # Data Quality Issues
    # ----------------------------
    duplicates = df.duplicated().sum()
    min_price_row = df.nsmallest(1, 'price_kes').iloc[0]
    max_price_row = df.nlargest(1, 'price_kes').iloc[0]
    size_outliers = df[(df['size_sqft'] < 100) & (df['bedrooms'] > 1)]
    size_outlier_str = ""
    if not size_outliers.empty:
        row = size_outliers.iloc[0]
        size_outlier_str = f"{row['size_sqft']} sqft properties with {row['bedrooms']} bedrooms - impossible"

    missing_sizes = df['size_sqft'].isnull().sum()
    missing_amenities = df['amenities'].isnull().sum()

    f.write("\n## Data Quality Issues Identified\n\n")
    f.write("### Critical Issues\n")
    f.write(f"- Duplicates: {duplicates} duplicate rows\n")
    f.write(f"- Price outliers: {min_price_row['price_kes']:,} KES property in {min_price_row['location']} - impossibly low\n")
    f.write(f"- Size outliers: {size_outlier_str}\n")
    f.write(f"- Missing values: {missing_sizes} missing sizes, {missing_amenities} missing amenities\n")

    # ----------------------------
    # Location Summary (Top 10)
    # ----------------------------
    location_summary = df.groupby('location').agg(
        count=('price_kes','count'),
        min_price=('price_kes','min'),
        max_price=('price_kes','max')
    ).sort_values('count', ascending=False).head(10)
    
    f.write("\n## Location Summary (Top 10 by count)\n\n")
    f.write(location_summary.to_markdown() + "\n\n")

    # ----------------------------
    # Property Type Breakdown
    # ----------------------------
    ptype_summary = df.groupby('property_type').agg(
        count=('price_kes','count'),
        avg_price=('price_kes','mean')
    )
    
    f.write("## Property Type Breakdown\n\n")
    f.write(ptype_summary.round(2).to_markdown() + "\n\n")

    # ----------------------------
    # Notes / Next Steps
    # ----------------------------
    f.write("## Notes\n")
    f.write("- Review and clean outliers in price and size.\n")
    f.write("- Fill missing amenities with 'None'.\n")
    f.write("- Consider engineered features: price_per_sqft, amenity_score, month from listing_date.\n")

print(f"Data dictionary saved as '{md_file}'")


Data dictionary saved as 'dictionary.md'
